# Climate Action Reserve
https://www.climateactionreserve.org

In [5]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

### Load data

In [6]:
df = pd.read_csv("../input/Climate_Action_Reserve/homogenised_dataset_CAR.csv")
df.head(1)

,site_id_created,project_id_reported,site_id_reported,site_description_reported,site_sqkm,trees_planted_reported,country,project_description_reported,planting_date_reported,survival_rate_reported,host_name,url,species_count_reported,species_planted_reported,geometry,Creator,project_id_created
0,NaN,CAR1957,NaN,NaN,NaN,NaN,US,The project consists of areas that were signif...,2/8/2023,NaN,Sierra Pacific Industries,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df["planting_date_type"]="Crediting Period Start Date"

### Save it

In [9]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   site_id_created               0 non-null      float64 
 1   project_id_reported           333 non-null    object  
 2   site_id_reported              0 non-null      float64 
 3   site_description_reported     0 non-null      float64 
 4   site_sqkm                     0 non-null      float64 
 5   trees_planted_reported        0 non-null      float64 
 6   country                       333 non-null    object  
 7   project_description_reported  333 non-null    object  
 8   planting_date_reported        333 non-null    object  
 9   survival_rate_reported        0 non-null      float64 
 10  host_name                     16 non-null     object  
 11  url                           78 non-null     object  
 12  species_count_reported        0 non-null  

In [8]:


gdf = gpd.GeoDataFrame(df)  
gdf.to_file('../midsave/climate_action_reserve.gpkg', driver='GPKG')